In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

In [3]:
mpl.matplotlib_fname()
font_dirs = ['D:\\Software\\Anaconda\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf' ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)
mpl.font_manager._rebuild()
result = font_manager.findfont("Rounded Mplus 1c")
print(result)
mpl.rc('font', family='Rounded Mplus 1c')

D:\Anaconda\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\MPLUSRounded1c-Regular.ttf


In [5]:
# df_merged = pd.read_csv("E:\\Projects\\KAIHO\\Data\\Kaiho_merged_data.csv")
df_merged = pd.read_csv("Kaiho_merged_data.csv")

In [6]:
list_col = list(df_merged.columns)
list_fil = list(filter(lambda col :  df_merged[col].nunique() > 1 ,list_col))
df_fil = df_merged[list_fil]
df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'] = df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'].apply(lambda x: x.replace(',', '')).astype('float64')
df_fil.drop(columns = ['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額.1'],inplace= True)

<ipython-input-6-af902f39b4fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'] = df_fil['Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額'].apply(lambda x: x.replace(',', '')).astype('float64')
D:\Anaconda\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
for col in list(df_fil.columns):
 (df_fil[col].dtype ==  'object') and (df_fil[col].fillna(value='unknown',inplace = True)) or (df_fil[col].fillna(value=0, inplace = True))

D:\Anaconda\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [8]:
df_fil.isnull().sum()

Vị trí_ロケーション                                           0
Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Nv sản xuất_生産担当者                                       0
Số nhập kho_入庫番号                                        0
Nguồn nhập hàng_仕入先                                     0
Mã loại phụ tùng_商品コード                                  0
Tên phụ tùng_商品名                                        0
Số nhập kho(phụ tùng)_入庫番号 (部品)                         0
Tên xe_車名                                               0
Model xe_車輌型式                                           0
Engine model_エンジン型式                                     0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá b

In [11]:
df_fil.groupby('Tên phụ tùng_商品名')['Tên phụ tùng_商品名'].count()

Tên phụ tùng_商品名
エンジン                14926
ノーズカット（個別）           3524
フェイスカット                 8
フロントアクスル（個別）          158
ミッション（個別）               2
リアアクスル（個別）            109
リアホーシング（デフ無）（個別）        1
外装コンプリート               13
ＣＫＤ／コンプリート           1207
ＣＫＤ／コンプリート（バイヤー）       22
Ｆアクスル（生/Ｄ）             69
Ｒアクスル（生/Ｄ）            389
Name: Tên phụ tùng_商品名, dtype: int64

In [12]:
df_fil.groupby('Tên phụ tùng_商品名')['Thành tích bán ra_販売実績'].sum()

Tên phụ tùng_商品名
エンジン                549620961.5
ノーズカット（個別）           37470500.0
フェイスカット                162000.0
フロントアクスル（個別）          1353000.0
ミッション（個別）               10000.0
リアアクスル（個別）             787000.0
リアホーシング（デフ無）（個別）         8000.0
外装コンプリート              1087450.0
ＣＫＤ／コンプリート          166435819.0
ＣＫＤ／コンプリート（バイヤー）      1710000.0
Ｆアクスル（生/Ｄ）             351500.0
Ｒアクスル（生/Ｄ）            2472500.0
Name: Thành tích bán ra_販売実績, dtype: float64

In [66]:
df_best_phutung = df_fil[(df_fil['Tên phụ tùng_商品名']=='エンジン') | (df_fil['Tên phụ tùng_商品名']=='ノーズカット（個別）') |(df_fil['Tên phụ tùng_商品名']== 'ＣＫＤ／コンプリート')]
df_best_phutung

,Vị trí_ロケーション,Ngày sản xuất_生産日,Tháng sản xuất_生産月,Năm sản xuất_生産年,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),...,Tháng mua hàng_仕入月,Năm mua_仕入年,Chi tiết xếp hạng_ランク詳細,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード,Số body_車体番号,CONTAINER_CONTAINER,Tên khách hàng_売上先名,D/G_Ｄ／Ｇ,Đang tải nguồn_積込元
0,会宝産業本社,17,11,2015,山下大輝,'0000000302676,西田 敏勝,'110010,ノーズカット（個別）,'0000007988203,...,12.0,2012.0,unknown,39332,990010.0,'1001019,unknown,"一番オート OOO ""LIGHT VL""",G,会宝産業本社
1,会宝産業本社,3,2,2015,中嶋 修,'0000000303771,ｴｻﾞｼ自動車,'100000,エンジン,'0000007739581,...,1.0,2015.0,unknown,24925,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,G,会宝産業本社
2,会宝産業本社,3,2,2015,楊 志毅,'0000000303771,ｴｻﾞｼ自動車,'110010,ノーズカット（個別）,'0000007712751,...,1.0,2015.0,unknown,24925,990000.0,'6062039,40HC,木戸クンク・ジャイブ（ＥＫ３）,G,会宝産業本社
3,会宝産業本社,5,2,2015,西川航平,'0000000314958,ＪＵ石川,'110010,ノーズカット（個別）,'0000007732276,...,1.0,2015.0,@,24945,319800.0,'2010272,40HC,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
4,会宝産業本社,25,2,2015,中里太明,'0000000319165,会宝パーツサービス㈲福井店 アップガレージ,'110010,ノーズカット（個別）,'0000007742703,...,2.0,2015.0,unknown,25365,208750.0,'072307,40HC,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20401,会宝産業本社,26,9,2013,中嶋 修,'0000000314082,ＧＭＳコーポレーシ(ｶｰｸﾘｴｲﾄ金沢),'100000,エンジン,'0000007353787,...,9.0,2013.0,unknown,19028,200460.0,'124900,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20402,会宝産業本社,30,9,2013,菊池忠義,'0000000314150,㈱日産プリンス東大通店,'100000,エンジン,'0000007356752,...,9.0,2013.0,unknown,19028,511610.0,'201247,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20403,会宝産業本社,28,9,2013,坂本光司,'0000000314364,JU岐阜 羽島オークション,'100000,エンジン,'0000007355332,...,9.0,2013.0,@,19028,310490.0,'0044142,20,㈱パシフィック・トレーディング・インターナショナル,G,会宝産業本社
20405,会宝産業本社,28,2,2012,菊池忠義,'0000000287252,Ｋ・Ｓ・Ｋ,'110010,ノーズカット（個別）,'0000005894104,...,2.0,2012.0,以前のバラし分,20406,208900.0,'7063020,20,FADEEV EVGENY,G,会宝産業本社


In [67]:
cat_col =  list(df_best_phutung.select_dtypes(include='object'))
df_cat  = df_best_phutung[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [68]:
num_col = list(df_best_phutung.select_dtypes(include=['int64','float64']))
df_num = df_best_phutung[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [69]:
df_target = df_best_phutung['Thành tích bán ra_販売実績']
df_target

0        12000.0
1        30000.0
2        10000.0
3        12000.0
4        10000.0
          ...   
20401    25000.0
20402    20000.0
20403    30000.0
20405    12000.0
20423    12000.0
Name: Thành tích bán ra_販売実績, Length: 19657, dtype: float64

In [70]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))

In [71]:
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,1,22,3831,2444,2,1,15680,269,365,383,...,12000.0,12000,30,3,2016,22.0,12.0,2012.0,39332,990010.0
1,1,5,3943,2757,1,0,14310,210,72,153,...,30000.0,30000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
2,1,30,3943,2757,2,1,14147,210,72,153,...,10000.0,10000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
3,1,43,5443,2707,2,1,14269,286,1054,231,...,12000.0,12000,23,2,2015,15.0,1.0,2015.0,24945,319800.0
4,1,9,6078,803,2,1,14337,295,4,472,...,10000.0,10000,24,3,2015,23.0,2.0,2015.0,25365,208750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19652,1,5,5292,2702,1,0,11032,301,1244,418,...,0.0,0,8,10,2013,21.0,9.0,2013.0,19028,200460.0
19653,1,41,5303,518,1,0,11068,412,1123,286,...,0.0,0,8,10,2013,26.0,9.0,2013.0,19028,511610.0
19654,1,19,5357,82,1,0,11052,53,893,23,...,0.0,0,8,10,2013,25.0,9.0,2013.0,19028,310490.0
19655,1,41,1717,2709,2,1,7625,211,1196,70,...,0.0,0,6,2,2014,24.0,2.0,2012.0,20406,208900.0


In [72]:
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [73]:
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)

In [74]:
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)

In [75]:
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.3, random_state=42)

In [76]:
names = ["Gradient_Boosting", "Decision_Tree", "Extra_Trees", "Random_Forest","AdaBoost","XGBoost"]
models = [
    GradientBoostingRegressor(),
    DecisionTreeRegressor(max_depth=5),
    ExtraTreesRegressor(),
    RandomForestRegressor(),
    AdaBoostRegressor(),
    XGBRegressor()]

In [77]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[ 30681.54282532  27806.63411831  23856.03349457 ...  29120.36728822
  90475.38079198 100752.37904803]
[ 31559.47394541  26257.45243704  26257.45243704 ...  31559.47394541
  90398.51438849 104539.3030303 ]
[29485. 31000. 25400. ... 33030. 88252. 98080.]
[ 31870.    31070.    24040.   ...  30510.    92763.95 102950.6 ]
[71003.91084002 45195.00935016 47404.66896869 ... 49870.7714736
 99851.74642857 99851.74642857]
[ 30708.068  27461.13   25923.78  ...  31303.607  96544.484 103192.15 ]


In [78]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.935,9016.717,14.202
1,Decision_Tree,0.860,13205.648,17.531
2,Extra_Trees,0.951,7827.562,11.044
3,Random_Forest,0.926,9584.189,11.940
4,AdaBoost,0.328,28908.371,122.557
5,XGBoost,0.920,9980.719,12.388


In [79]:
shap_list = ['Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額',
            'Mã loại phụ tùng_商品コード',
            'Tổng giá nhập_仕入合計',
            'Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額',
            'Tên khách hàng_売上先名',
            'Engine model_エンジン型式',
            'Số phút công_標準工数',
            'Số nhập kho_入庫番号',
            'Khách hàng_仕向先']

In [81]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,-0.626250,1.852361,-0.758469,-0.667362,1.046165,1.168644,-0.920594,-0.994754,0.235536
1,-0.274946,-0.247516,0.053766,0.133472,1.244081,-0.329707,0.466135,-0.973251,1.108467
2,-0.626250,1.852361,-0.596022,-0.347029,1.244081,-0.329707,-0.920594,-0.973251,1.108467
3,-0.626250,1.852361,-0.732153,-0.667362,0.980193,0.178429,-0.920594,-0.685268,0.904783
4,-0.626250,1.852361,-0.732153,-0.667362,0.980193,1.748440,-0.920594,-0.563355,0.904783
...,...,...,...,...,...,...,...,...,...
19652,-0.274946,-0.247516,0.053766,0.133472,0.980193,1.396654,1.506182,-0.714258,0.904783
19653,-0.410063,-0.247516,-0.108681,-0.026695,0.980193,0.536731,0.466135,-0.712146,0.904783
19654,-0.139829,-0.247516,0.216213,0.293639,0.980193,-1.176601,-0.227230,-0.701779,0.904783
19655,-0.626250,1.852361,-0.719360,-0.667362,-0.932995,-0.870416,-0.920594,-1.400619,-0.986566


In [82]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.3, random_state=42)

In [83]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[ 29594.54397492  27901.32059778  24890.73094814 ...  29836.87667836
  90709.49904932 100143.97364593]
[ 31559.47394541  26257.45243704  26257.45243704 ...  31559.47394541
  79664.97540984 104218.91      ]
[30070.  28320.  24840.  ... 36970.  89349.8 98797.7]
[29850.  28120.  24230.  ... 33420.  89609.7 99929.2]
[71906.12313982 61910.07833671 51338.68142713 ... 61910.07833671
 90924.87772854 98446.42139434]
[ 29899.67   28709.95   22928.898 ...  31251.088  92370.15  100541.38 ]


In [84]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.937,8871.384,14.480
1,Decision_Tree,0.812,15285.420,17.249
2,Extra_Trees,0.946,8210.253,11.411
3,Random_Forest,0.936,8928.370,11.653
4,AdaBoost,0.227,30989.949,129.725
5,XGBoost,0.935,8962.946,12.375


# Phu tung エンジン

In [85]:
df_エンジン = df_fil[(df_fil['Tên phụ tùng_商品名']=='エンジン')]

In [86]:
cat_col =  list(df_エンジン.select_dtypes(include='object'))
df_cat  = df_エンジン[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [87]:
num_col = list(df_エンジン.select_dtypes(include=['int64','float64']))
df_num = df_エンジン[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [88]:
df_target = df_エンジン['Thành tích bán ra_販売実績']
df_target

1        30000.0
6        25000.0
8        45000.0
10       30000.0
11       85000.0
          ...   
20384    15000.0
20400    25000.0
20401    25000.0
20402    20000.0
20403    30000.0
Name: Thành tích bán ra_販売実績, Length: 14926, dtype: float64

In [89]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,0,4,3209,2344,0,0,10770,194,62,141,...,30000.0,30000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
1,0,20,5018,1388,0,0,11083,243,797,104,...,25000.0,25000,30,4,2015,15.0,4.0,2015.0,25896,518170.0
2,0,20,5277,2302,0,0,10736,286,314,125,...,45000.0,45000,23,2,2015,5.0,4.0,2014.0,24945,319800.0
3,0,20,5449,263,0,0,11408,194,62,141,...,30000.0,30000,31,8,2015,3.0,4.0,2014.0,37373,304510.0
4,0,17,5450,377,0,0,10690,156,1405,28,...,85000.0,85000,14,1,2015,20.0,12.0,2014.0,24515,701400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14921,0,4,12139,1618,0,0,2358,21,545,332,...,0.0,0,11,4,2019,30.0,3.0,2019.0,52628,990010.0
14922,0,17,4273,30,0,0,8126,361,1330,389,...,0.0,0,8,10,2013,24.0,9.0,2013.0,19028,704400.0
14923,0,4,4322,2297,0,0,8130,280,1152,389,...,0.0,0,8,10,2013,21.0,9.0,2013.0,19028,200460.0
14924,0,36,4333,468,0,0,8166,384,1037,268,...,0.0,0,8,10,2013,26.0,9.0,2013.0,19028,511610.0


In [90]:
ss = StandardScaler()
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.3, random_state=42)

In [91]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[24556.44751736 64113.76096631 32099.7329872  ... 61424.49787734
 45696.18179041 22241.63461725]
[22624.57542857 65160.56105611 27280.1659306  ... 65160.56105611
 44815.92039801 22624.57542857]
[26490. 61340. 30450. ... 61265. 44520. 20920.]
[26100. 66875. 31790. ... 62360. 48280. 21140.]
[32564.17492206 63143.44812304 40756.67655786 ... 63143.44812304
 62497.70963855 33476.83697984]
[26505.87  72040.734 34609.99  ... 60156.207 42517.36  22383.146]


In [92]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.858,8235.718,13.864
1,Decision_Tree,0.785,10130.730,17.336
2,Extra_Trees,0.871,7845.480,12.097
3,Random_Forest,0.856,8294.245,12.554
4,AdaBoost,0.469,15936.603,46.986
5,XGBoost,0.872,7824.688,12.044


In [93]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,-0.354034,0.0,0.096008,0.093054,1.119903,-0.352448,0.050981,-0.987174,1.087427
1,-0.354034,0.0,-0.422123,-0.950239,1.119903,-0.613234,1.099833,-0.567332,1.087427
2,-0.121480,0.0,0.372345,0.301713,0.983141,-0.465220,-0.997871,-0.507222,1.196601
3,-0.121480,0.0,0.664506,-0.950239,-0.042570,-0.352448,0.050981,-0.467303,0.687121
4,1.273844,0.0,2.030366,1.553665,0.983141,-1.148903,-0.997871,-0.467071,1.196601
...,...,...,...,...,...,...,...,...,...
14921,-0.819142,0.0,-1.103447,-0.950239,-1.410184,0.993773,-0.997871,1.085346,-1.496363
14922,-0.354034,0.0,0.096008,0.093054,0.983141,1.395525,0.050981,-0.740235,0.832687
14923,-0.354034,0.0,0.096008,0.093054,0.983141,1.395525,1.624258,-0.728863,0.832687
14924,-0.586588,0.0,-0.180329,-0.115604,0.983141,0.542683,0.050981,-0.726310,0.832687


In [94]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.3, random_state=42)

In [95]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[23616.55314273 62501.79324465 33774.07147083 ... 58887.94270912
 42478.37317807 21089.65356843]
[22626.00630011 55672.02608696 28140.27124464 ... 65514.21800948
 44815.92039801 22626.00630011]
[26970.   62760.14 31820.   ... 61400.   42390.   20050.  ]
[26970.   63472.69 31040.   ... 58500.   44020.   19930.  ]
[32792.14094351 65473.96072013 39682.71097372 ... 65473.96072013
 56810.7618198  32792.14094351]
[25324.156 65633.73  35484.168 ... 53191.2   42749.074 20035.062]


In [96]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.853,8375.430,14.362
1,Decision_Tree,0.800,9771.604,16.827
2,Extra_Trees,0.860,8180.122,12.508
3,Random_Forest,0.857,8264.101,12.258
4,AdaBoost,0.355,17567.893,48.647
5,XGBoost,0.867,7985.271,12.175


# Phu tung ノーズカット（個別）

In [98]:
df_ノーズカット = df_fil[(df_fil['Tên phụ tùng_商品名']=='ノーズカット（個別）')]

In [99]:
cat_col =  list(df_ノーズカット.select_dtypes(include='object'))
df_cat  = df_ノーズカット[cat_col]
df_cat.isnull().sum()

Vị trí_ロケーション                      0
Nv sản xuất_生産担当者                  0
Số nhập kho_入庫番号                   0
Nguồn nhập hàng_仕入先                0
Mã loại phụ tùng_商品コード             0
Tên phụ tùng_商品名                   0
Số nhập kho(phụ tùng)_入庫番号 (部品)    0
Tên xe_車名                          0
Model xe_車輌型式                      0
Engine model_エンジン型式                0
Khách hàng_仕向先                     0
Truyền động 1_駆動1                  0
TM_TM                              0
Truyền động 2_駆動2                  0
Chi tiết_詳細                        0
Rank_ランク                           0
Loại xe_車種                         0
Nhà sản xuất_メーカー                  0
Chi tiết xếp hạng_ランク詳細            0
Số body_車体番号                       0
CONTAINER_CONTAINER                0
Tên khách hàng_売上先名                0
D/G_Ｄ／Ｇ                            0
Đang tải nguồn_積込元                 0
dtype: int64

In [100]:
num_col = list(df_ノーズカット.select_dtypes(include=['int64','float64']))
df_num = df_ノーズカット[num_col]
df_num.isnull().sum()

Ngày sản xuất_生産日                                       0
Tháng sản xuất_生産月                                      0
Năm sản xuất_生産年                                        0
Tổng giá nhập_仕入合計                                      0
Số phút công_標準工数                                       0
Kết quả_生産高                                             0
Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額    0
Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額     0
Số lượng bán ra_販売数量                                    0
Thành tích bán ra_販売実績                                  0
Số km đã chạy_走行距離                                      0
Năm sản xuất_年式                                         0
Số chỉ định loại_型式指定番号                                 0
Số phân loại danh mục_類別区分番号                            0
Đơn giá mua_仕入単価                                        0
Giờ làm việc_工数                                         0
Đơn giá bán hàng_売上単価                                   0
Sản lượng bán 

In [101]:
df_target = df_ノーズカット['Thành tích bán ra_販売実績']
df_target

0        12000.0
2        10000.0
3        12000.0
4        10000.0
5         9000.0
          ...   
20329    12000.0
20334    10000.0
20342    10000.0
20405    12000.0
20423    12000.0
Name: Thành tích bán ra_販売実績, Length: 3524, dtype: float64

In [102]:
le = LabelEncoder()
df_cat_encoded = df_cat.apply(lambda col: le.fit_transform(col))
df_encode = pd.merge(df_cat_encoded, df_num, on = df_cat_encoded.index, how='outer')
df_encode.drop(columns = ['key_0'], inplace=True)
df_encode

,Vị trí_ロケーション,Nv sản xuất_生産担当者,Số nhập kho_入庫番号,Nguồn nhập hàng_仕入先,Mã loại phụ tùng_商品コード,Tên phụ tùng_商品名,Số nhập kho(phụ tùng)_入庫番号 (部品),Tên xe_車名,Model xe_車輌型式,Engine model_エンジン型式,...,Đơn giá bán hàng_売上単価,Sản lượng bán ra_売上金額,Ngày bán hàng_売上日,Tháng bán hàng_売上月,Năm bán hàng_売上年,Ngày mua_仕入日,Tháng mua hàng_仕入月,Năm mua_仕入年,Số phiếu_伝票番号,Mã nhà cung cấp_仕入先コード
0,0,12,862,782,0,0,2930,156,187,193,...,12000.0,12000,30,3,2016,22.0,12.0,2012.0,39332,990010.0
1,0,20,886,880,0,0,2660,118,37,86,...,10000.0,10000,19,2,2015,31.0,1.0,2015.0,24925,990000.0
2,0,30,1279,852,0,0,2688,166,514,117,...,12000.0,12000,23,2,2015,15.0,1.0,2015.0,24945,319800.0
3,0,5,1410,351,0,0,2710,172,1,242,...,10000.0,10000,24,3,2015,23.0,2.0,2015.0,25365,208750.0
4,0,33,1411,568,0,0,2718,114,167,94,...,9000.0,9000,26,5,2015,24.0,3.0,2015.0,26148,518170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3519,0,16,406,689,0,0,1270,136,341,5,...,0.0,0,30,11,2013,7.0,12.0,2011.0,19680,990010.0
3520,0,33,2369,607,0,0,3202,26,288,190,...,0.0,0,30,8,2017,27.0,2.0,2017.0,45183,990010.0
3521,0,20,2581,268,0,0,37,57,280,2,...,0.0,0,13,12,2017,6.0,12.0,2017.0,46320,312800.0
3522,0,29,364,854,0,0,1352,119,550,37,...,0.0,0,6,2,2014,24.0,2.0,2012.0,20406,208900.0


In [103]:
ss = StandardScaler()
df_scale = pd.DataFrame(ss.fit_transform(df_encode),columns = df_encode.columns)
df_scale.drop(columns=['Thành tích bán ra_販売実績','Đơn giá bán hàng_売上単価','Sản lượng bán ra_売上金額'],inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df_scale, df_target, test_size=0.3, random_state=42)

In [104]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[11716.15325801 11732.19156479  8568.3820443  ...  8174.05850865
 10615.79178223 10958.61969981]
[10297.16312057 11700.91027308 10297.16312057 ...  8103.09278351
 10297.16312057 10297.16312057]
[11670. 12020.  8910. ...  8000. 11860. 11210.]
[13090. 11960.  8870. ...  8000. 11940. 11140.]
[11591.86746988 12149.36708861 10886.3760218  ... 10941.29287599
 10770.74235808 11803.83480826]
[12552.57  12020.556  8115.708 ...  9832.67  11613.021 13171.067]


In [105]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,0.287,2265.033,11.108
1,Decision_Tree,0.076,2577.357,12.811
2,Extra_Trees,0.286,2265.777,9.054
3,Random_Forest,0.060,2600.207,9.389
4,AdaBoost,-0.766,3564.357,20.117
5,XGBoost,-0.402,3175.065,10.069


In [106]:
df_shap = df_scale[shap_list]
df_shap

,Ước tính giá bán ra tại thời điểm sản xuất_生産時販売予測額,Mã loại phụ tùng_商品コード,Tổng giá nhập_仕入合計,Ước tính giá bán ra tại thời điểm nhập hàng_仕入時販売予測額,Tên khách hàng_売上先名,Engine model_エンジン型式,Số phút công_標準工数,Số nhập kho_入庫番号,Khách hàng_仕向先
0,0.015365,0.0,-0.836068,-0.212885,0.993177,1.072949,0.730873,-0.884211,0.046482
1,0.015365,0.0,1.136016,4.487666,1.203859,-0.281394,0.730873,-0.860619,0.915461
2,0.015365,0.0,-0.516591,-0.212885,0.887836,0.110986,0.730873,-0.474299,0.752527
3,0.015365,0.0,-0.516591,-0.212885,0.887836,1.693163,0.730873,-0.345525,0.752527
4,0.015365,0.0,-0.536558,-0.212885,0.887836,-0.180135,0.730873,-0.344542,0.752527
...,...,...,...,...,...,...,...,...,...
3519,0.015365,0.0,-0.443377,-0.212885,-1.008303,-1.306645,0.730873,-1.332460,-1.202675
3520,-0.090775,0.0,-0.836068,-0.212885,-0.481598,1.034977,-1.368227,0.597174,-0.822497
3521,-0.090775,0.0,0.415214,-0.212885,-1.008303,-1.344617,-1.368227,0.805571,-1.202675
3522,0.015365,0.0,-0.361289,-0.212885,-1.008303,-0.901607,0.730873,-1.373746,-1.202675


In [107]:
x_train, x_test, y_train, y_test = train_test_split(df_shap, df_target, test_size=0.3, random_state=42)

In [108]:
rmse = []
mape = []
r_squared = []
for  model in models:
    model.fit(x_train, y_train)
    y_pred  = model.predict(x_test)
    print(y_pred)
    rmse_score = mean_squared_error(y_test, y_pred,squared=False)
    mape_score = np.mean(np.abs((y_test.values - y_pred)/y_test.values)*100)
    r_squared_score  = model.score(x_test,y_test)
    r_squared.append(r_squared_score)
    rmse.append(rmse_score)
    mape.append(mape_score)

[12273.37314375 11421.1644885   9177.94567818 ...  8126.74026651
 11510.3853851  10948.93543652]
[11750.         11489.32676519 10281.1550152  ...  8057.47126437
 10281.1550152  10281.1550152 ]
[14730. 11980.  8370. ...  8000. 11980. 13230.]
[13350. 11720.  8550. ...  8000. 11940. 10980.]
[11766.22222222 11915.96638655 11766.22222222 ... 11240.31190926
 11945.98262758 11830.2891933 ]
[15077.208  11769.617   9888.17   ...  7960.6685 11861.41   12473.025 ]


In [109]:
df_score = pd.DataFrame()
df_score['MODELS'] = names
df_score['R_SQUARED'] = r_squared
df_score['RMSE'] = rmse
df_score['MAPE'] = mape
df_score['RMSE'] = round(df_score['RMSE'],3)
df_score['MAPE'] = round(df_score['MAPE'],3)
df_score['R_SQUARED'] = round(df_score['R_SQUARED'],3)

df_score

,MODELS,R_SQUARED,RMSE,MAPE
0,Gradient_Boosting,-0.223,2965.620,12.036
1,Decision_Tree,-0.146,2870.974,13.245
2,Extra_Trees,-0.658,3453.033,10.896
3,Random_Forest,-0.424,3199.879,11.370
4,AdaBoost,-0.458,3238.684,22.281
5,XGBoost,-0.612,3404.727,11.781
